# This project demonstrate how to use SQL with python.
In the project am going to demonstrate how to use python and SQL to query, update, and modify the data in an sqlite3 database. The data set used for this project can be found [here](https://github.com/fivethirtyeight/data/tree/master/college-majors).Here are the descriptions for the columns in the preview:

 - Rank - The major's rank by median earnings
 - Major_code - The major's code or ID
 - Major - The name of the major
 - Major_category - The broader category the major belongs to
 - Total - The total number of people who studied the major
 - Sample_size - The sample size (unweighted) of graduates with full time jobs
 - Men - The number of male graduates
 - Women - The number of female graduates
 - ShareWomen - Women as a proportion of the total number of graduates
 - Employed - The number of employed graduates

# SQL SELECT syntax
The SELECT syntax statement can be written as:

SELECT [column1, column2, ...] FROM tableName;

To use python to query the database using the above SELECT statement involves the following steps:
 - import the python database drive for the database you are using,
 - Connect to the database using [connect() function](https://docs.python.org/3/library/sqlite3.html#sqlite3.connect)
 - Use [Cursor class](https://docs.python.org/3/library/sqlite3.html#cursor-objects) to:
     -  Run a query against the database
     - Parse the results from the database
     - Convert the results to native Python objects
     - Store the results within the Cursor instance as a local variable

After running a query and converting the results to a list of tuples, the Cursor instance stores the list as a local variable. This can be done as shown below.


In [5]:
import sqlite3

conn = sqlite3.connect("jobs.db")
cursor = conn.cursor()
query1 = "SELECT Rank, Major FROM recent_grads"
cursor.execute(query1)
data = cursor.fetchall()
print(data[0:20])

[(1, 'PETROLEUM ENGINEERING'), (2, 'MINING AND MINERAL ENGINEERING'), (3, 'METALLURGICAL ENGINEERING'), (4, 'NAVAL ARCHITECTURE AND MARINE ENGINEERING'), (5, 'CHEMICAL ENGINEERING'), (6, 'NUCLEAR ENGINEERING'), (7, 'ACTUARIAL SCIENCE'), (8, 'ASTRONOMY AND ASTROPHYSICS'), (9, 'MECHANICAL ENGINEERING'), (10, 'ELECTRICAL ENGINEERING'), (11, 'COMPUTER ENGINEERING'), (12, 'AEROSPACE ENGINEERING'), (13, 'BIOMEDICAL ENGINEERING'), (14, 'MATERIALS SCIENCE'), (15, 'ENGINEERING MECHANICS PHYSICS AND SCIENCE'), (16, 'BIOLOGICAL ENGINEERING'), (17, 'INDUSTRIAL AND MANUFACTURING ENGINEERING'), (18, 'GENERAL ENGINEERING'), (19, 'ARCHITECTURAL ENGINEERING'), (20, 'COURT REPORTING')]


So far, we've been running queries by creating a Cursor instance, and then calling the execute method on the instance. The SQLite library actually allows us to skip creating a Cursor altogether by using the execute method within the Connection object itself. SQLite will create a Cursor instance for us under the hood and our query run against the database, but this shortcut allows us to skip a step. Here's what the code looks like:

In [7]:
conn = sqlite3.connect("jobs.db")
query = "select * from recent_grads;"
data = conn.execute(query).fetchall()
print(data[0:5])

[(0, 1, 2419, 'PETROLEUM ENGINEERING', 'Engineering', 2339, 36, 2057, 282, 0.120564344, 1976, 1849, 270, 1207, 37, 0.018380527, 110000, 95000, 125000, 1534, 364, 193), (1, 2, 2416, 'MINING AND MINERAL ENGINEERING', 'Engineering', 756, 7, 679, 77, 0.10185185199999999, 640, 556, 170, 388, 85, 0.117241379, 75000, 55000, 90000, 350, 257, 50), (2, 3, 2415, 'METALLURGICAL ENGINEERING', 'Engineering', 856, 3, 725, 131, 0.153037383, 648, 558, 133, 340, 16, 0.024096386, 73000, 50000, 105000, 456, 176, 0), (3, 4, 2417, 'NAVAL ARCHITECTURE AND MARINE ENGINEERING', 'Engineering', 1258, 16, 1123, 135, 0.107313196, 758, 1069, 150, 692, 40, 0.050125313, 70000, 43000, 80000, 529, 102, 0), (4, 5, 2405, 'CHEMICAL ENGINEERING', 'Engineering', 32260, 289, 21239, 11021, 0.341630502, 25694, 23170, 5180, 16697, 1672, 0.061097712, 65000, 50000, 75000, 18314, 4440, 972)]


# Cursor
To make it easier to work with large results sets, the Cursor class allows us to control the number of results we want to retrieve at any given time. To return a single result (as a tuple), we use the Cursor method fetchone(). To return n results, we use the Cursor method fetchmany() as shown below.

In [8]:
query = "SELECT Major,Major_category FROM recent_grads"
five_results = conn.execute(query).fetchmany(5)
five_results

[('PETROLEUM ENGINEERING', 'Engineering'),
 ('MINING AND MINERAL ENGINEERING', 'Engineering'),
 ('METALLURGICAL ENGINEERING', 'Engineering'),
 ('NAVAL ARCHITECTURE AND MARINE ENGINEERING', 'Engineering'),
 ('CHEMICAL ENGINEERING', 'Engineering')]

# Close()
To close a connection to a database, use the Connection instance method close(). After closing the connection, attempting to query the database using any linked Cursor instances will return the following error as shown below:

In [10]:
conn.close()
data = conn.execute(query).fetchall()

ProgrammingError: Cannot operate on a closed database.

# WHERE Clause
To filter rows by specific criteria, we need to use the WHERE statement.
The WHERE statement requires three things:

 - The column we want the database to filter on
 - A comparison operator that specifies how we want to compare a value in a column
 - The value we want the database to compare each value to.

Here are the comparison operators we can use:

 - Less than: <
 - Less than or equal to: <=
 - Greater than: >
 - Greater than or equal to: >=
 - Equal to: =
 - Not equal to: !=
To domonstrate WHERE clause the query below is used to fetch Major,Major_category  in the recent_grads table where shareWomen is greater than 50%

In [16]:
conn = sqlite3.connect("jobs.db")
data = conn.execute("SELECT Major,Major_category  FROM recent_grads WHERE ShareWomen > 0.5;").fetchall()
data[0:10]

[('ACTUARIAL SCIENCE', 'Business'),
 ('COMPUTER SCIENCE', 'Computers & Mathematics'),
 ('ENVIRONMENTAL ENGINEERING', 'Engineering'),
 ('NURSING', 'Health'),
 ('INDUSTRIAL PRODUCTION TECHNOLOGIES', 'Engineering'),
 ('COMPUTER AND INFORMATION SYSTEMS', 'Computers & Mathematics'),
 ('INFORMATION SCIENCES', 'Computers & Mathematics'),
 ('APPLIED MATHEMATICS', 'Computers & Mathematics'),
 ('PHARMACOLOGY', 'Biology & Life Science'),
 ('OCEANOGRAPHY', 'Physical Sciences')]

# LIMIT
Many queries return a large number of results, which can be cumbersome to work with. SQL comes with a statement called LIMIT that allows us to specify how many results we'd like the database to return as an integer value. LIMIT can be used to get 10 result from the query above as shown below without the need for subsetting while displaying the results.

In [17]:
data = conn.execute("SELECT Major,Major_category  FROM recent_grads WHERE ShareWomen > 0.5 LIMIT 10").fetchall()
data

[('ACTUARIAL SCIENCE', 'Business'),
 ('COMPUTER SCIENCE', 'Computers & Mathematics'),
 ('ENVIRONMENTAL ENGINEERING', 'Engineering'),
 ('NURSING', 'Health'),
 ('INDUSTRIAL PRODUCTION TECHNOLOGIES', 'Engineering'),
 ('COMPUTER AND INFORMATION SYSTEMS', 'Computers & Mathematics'),
 ('INFORMATION SCIENCES', 'Computers & Mathematics'),
 ('APPLIED MATHEMATICS', 'Computers & Mathematics'),
 ('PHARMACOLOGY', 'Biology & Life Science'),
 ('OCEANOGRAPHY', 'Physical Sciences')]

# Logical operators
Logical operators are keywords we can use to combine filtering criteria and express more specific conditions. Here are the two basic logical operators we use most often:

 - OR (returns either Condition1 or Condition2)
 - AND (returns both Condition1 and Condition2)
 
The Logical operators are used in the WHERE Clause. The AND syntax is shown below:

SELECT [column1, column2,...] FROM [table1]
WHERE [condition1] AND [condition2]



In [23]:
# AND
data = conn.execute('''
           SELECT Major,ShareWomen,Employed FROM recent_grads 
           WHERE ShareWomen > 0.5 AND Employed>10000 
           LIMIT 10''').fetchall()
data

[('COMPUTER SCIENCE', 0.578766338, 102087),
 ('NURSING', 0.896018988, 180903),
 ('COMPUTER AND INFORMATION SYSTEMS', 0.7077185020000001, 28459),
 ('INTERNATIONAL RELATIONS', 0.632986838, 21190),
 ('AGRICULTURE PRODUCTION AND MANAGEMENT', 0.59420765, 12323),
 ('CHEMISTRY', 0.5051405379999999, 48535),
 ('BUSINESS MANAGEMENT AND ADMINISTRATION', 0.580948004, 276234),
 ('BIOCHEMICAL SCIENCES', 0.515406449, 25678),
 ('HUMAN RESOURCES AND PERSONNEL MANAGEMENT', 0.672161443, 20760),
 ('MISCELLANEOUS HEALTH MEDICAL PROFESSIONS', 0.702020202, 10076)]

In [24]:
# OR
data = conn.execute('''
           SELECT Major,Median,Unemployed 
           FROM recent_grads
           WHERE median >= 10000 OR Unemployed <=1000
           LIMIT 20;''').fetchall()
data

[('PETROLEUM ENGINEERING', 110000, 37),
 ('MINING AND MINERAL ENGINEERING', 75000, 85),
 ('METALLURGICAL ENGINEERING', 73000, 16),
 ('NAVAL ARCHITECTURE AND MARINE ENGINEERING', 70000, 40),
 ('CHEMICAL ENGINEERING', 65000, 1672),
 ('NUCLEAR ENGINEERING', 65000, 400),
 ('ACTUARIAL SCIENCE', 62000, 308),
 ('ASTRONOMY AND ASTROPHYSICS', 62000, 33),
 ('MECHANICAL ENGINEERING', 60000, 4650),
 ('ELECTRICAL ENGINEERING', 60000, 3895),
 ('COMPUTER ENGINEERING', 60000, 2275),
 ('AEROSPACE ENGINEERING', 60000, 794),
 ('BIOMEDICAL ENGINEERING', 60000, 1019),
 ('MATERIALS SCIENCE', 60000, 78),
 ('ENGINEERING MECHANICS PHYSICS AND SCIENCE', 58000, 23),
 ('BIOLOGICAL ENGINEERING', 57100, 589),
 ('INDUSTRIAL AND MANUFACTURING ENGINEERING', 57000, 699),
 ('GENERAL ENGINEERING', 56000, 2859),
 ('ARCHITECTURAL ENGINEERING', 54000, 170),
 ('COURT REPORTING', 54000, 11)]

# Complex logical operation
Parenthese can be used to combine AND and OR operator in other to filter more complex query. For example assume  we want to  write a query that returned all Engineering majors that either had mostly female graduates or an unemployment rate below 5.1%, we would need to use parentheses to express this more complex logic. The three raw conditions we'll need are:

 - Major_category = 'Engineering'
 - ShareWomen >= 0.5
 - Unemployment_rate < 0.051

The SQl to achieve this is shown below.

In [25]:
data = conn.execute('''
           SELECT Major, Major_category, ShareWomen, Unemployment_rate
           FROM recent_grads
           WHERE (Major_category = 'Engineering') AND (ShareWomen > 0.5 OR Unemployment_rate < 0.051)''').fetchall()
data

[('PETROLEUM ENGINEERING', 'Engineering', 0.120564344, 0.018380527),
 ('METALLURGICAL ENGINEERING', 'Engineering', 0.153037383, 0.024096386),
 ('NAVAL ARCHITECTURE AND MARINE ENGINEERING',
  'Engineering',
  0.107313196,
  0.050125313),
 ('MATERIALS SCIENCE', 'Engineering', 0.310820285, 0.023042836),
 ('ENGINEERING MECHANICS PHYSICS AND SCIENCE',
  'Engineering',
  0.183985189,
  0.006334343),
 ('INDUSTRIAL AND MANUFACTURING ENGINEERING',
  'Engineering',
  0.34347321799999997,
  0.042875544),
 ('MATERIALS ENGINEERING AND MATERIALS SCIENCE',
  'Engineering',
  0.292607004,
  0.027788805),
 ('ENVIRONMENTAL ENGINEERING', 'Engineering', 0.558548009, 0.093588575),
 ('INDUSTRIAL PRODUCTION TECHNOLOGIES',
  'Engineering',
  0.75047259,
  0.028308097),
 ('ENGINEERING AND INDUSTRIAL MANAGEMENT',
  'Engineering',
  0.174122505,
  0.03365166)]

# IN Operator
The IN operation allows you to write a qury that fetch data that matches values in a given list.  It is used to help reduce the need for multiple OR conditions in a SELECT, INSERT, UPDATE, or DELETE statement. The syntax is as shown below:

expression IN (value1, value2, .... value_n);

Assume we want to  find all that meet all of the following criteria:
 - Major_category of Business or Arts or Health
 - Employed students greater than 20,000 or Unemployment_rate below 5.1%
The IN operator can be used to avoid using multiply OR condition in the WHERE clause as shown below.

In [26]:
data = conn.execute('''
           SELECT Major,Major_category,Employed,Unemployment_rate
           FROM recent_grads
           WHERE Major_category 
           IN ('Business', 'Arts', 'Health') AND (Employed > 20000 or Unemployment_rate < 0.051)''').fetchall()
data

[('OPERATIONS LOGISTICS AND E-COMMERCE',
  'Business',
  10027,
  0.047858702999999995),
 ('NURSING', 'Health', 180903, 0.04486272400000001),
 ('FINANCE', 'Business', 145696, 0.060686356),
 ('ACCOUNTING', 'Business', 165527, 0.069749014),
 ('MEDICAL TECHNOLOGIES TECHNICIANS', 'Health', 13150, 0.03698279),
 ('MEDICAL ASSISTING SERVICES', 'Health', 9168, 0.042506527),
 ('GENERAL BUSINESS', 'Business', 190183, 0.072861468),
 ('BUSINESS MANAGEMENT AND ADMINISTRATION',
  'Business',
  276234,
  0.07221834099999999),
 ('MARKETING AND MARKETING RESEARCH',
  'Business',
  178862,
  0.061215064000000007),
 ('HUMAN RESOURCES AND PERSONNEL MANAGEMENT', 'Business', 20760, 0.059569649),
 ('COMMERCIAL ART AND GRAPHIC DESIGN', 'Arts', 83483, 0.096797577),
 ('TREATMENT THERAPY PROFESSIONS', 'Health', 37861, 0.059821207),
 ('HOSPITALITY MANAGEMENT', 'Business', 36728, 0.061169193),
 ('GENERAL MEDICAL AND HEALTH SERVICES', 'Health', 24406, 0.082101621),
 ('FILM VIDEO AND PHOTOGRAPHIC ARTS', 'Arts', 3143

# ORDER BY
ORDER BY clause is used to sort the records in the result set for a SELECT statement. The syntax is shown below.

SELECT expressions
FROM tables
[WHERE conditions]
ORDER BY expression [ ASC | DESC ];

ASC Optional. ASC sorts the result set in ascending order by expression. This is the default behavior, if no modifier is provider. 
DESC Optional. DESC sorts the result set in descending order by expression.

In [28]:
conn.execute('''
    SELECT Major
    FROM recent_grads
    ORDER BY Major DESC 
    LIMIT 10''').fetchall()

[('ZOOLOGY',),
 ('VISUAL AND PERFORMING ARTS',),
 ('UNITED STATES HISTORY',),
 ('TREATMENT THERAPY PROFESSIONS',),
 ('TRANSPORTATION SCIENCES AND TECHNOLOGIES',),
 ('THEOLOGY AND RELIGIOUS VOCATIONS',),
 ('TEACHER EDUCATION: MULTIPLE LEVELS',),
 ('STUDIO ARTS',),
 ('STATISTICS AND DECISION SCIENCE',),
 ('SPECIAL NEEDS EDUCATION',)]

In [29]:
# order by multiple columns
conn.execute('''
    SELECT Major_category,Median,Major
    FROM recent_grads
    ORDER BY Major asc,Median desc
    LIMIT 20''').fetchall()

[('Business', 45000, 'ACCOUNTING'),
 ('Business', 62000, 'ACTUARIAL SCIENCE'),
 ('Communications & Journalism', 35000, 'ADVERTISING AND PUBLIC RELATIONS'),
 ('Engineering', 60000, 'AEROSPACE ENGINEERING'),
 ('Agriculture & Natural Resources', 40000, 'AGRICULTURAL ECONOMICS'),
 ('Agriculture & Natural Resources',
  40000,
  'AGRICULTURE PRODUCTION AND MANAGEMENT'),
 ('Agriculture & Natural Resources', 30000, 'ANIMAL SCIENCES'),
 ('Humanities & Liberal Arts', 28000, 'ANTHROPOLOGY AND ARCHEOLOGY'),
 ('Computers & Mathematics', 45000, 'APPLIED MATHEMATICS'),
 ('Engineering', 54000, 'ARCHITECTURAL ENGINEERING'),
 ('Engineering', 40000, 'ARCHITECTURE'),
 ('Humanities & Liberal Arts', 35000, 'AREA ETHNIC AND CIVILIZATION STUDIES'),
 ('Education', 32100, 'ART AND MUSIC EDUCATION'),
 ('Humanities & Liberal Arts', 31000, 'ART HISTORY AND CRITICISM'),
 ('Physical Sciences', 62000, 'ASTRONOMY AND ASTROPHYSICS'),
 ('Physical Sciences', 35000, 'ATMOSPHERIC SCIENCES AND METEOROLOGY'),
 ('Biology & Li

# SQL Summary Statistics
There are several ways to do summary statistics using python and SQL. One approach would be to use python to query the database, retrieve the results and store it as a list and then write code to do the desired summary statistics. While this approach works, it requires quite a bit of code, and it's also fairly slow. Another way would be to do the summary statistics on the database using SQL.It's often advantageous to do these computations in the SQL database instead of a Python environment because it's faster to code and execute. 

We'll be working with factbook.db, a SQLite database containing information about every country in the world. We'll use a table in the file called facts. Each row in facts represents a single country and contains several columns, including:

 - name - The name of the country
 - area - The total land and sea area of the country
 - population - The country's population
 - birth_rate - The country's birth rate
 - created_at - The date the record was created
 - updated_at - The date the record was updated

In [32]:
import sqlite3

conn = sqlite3.connect("factbook.db")
facts = facts = conn.execute("SELECT * FROM facts").fetchall()
facts[0:2]

[(1,
  'af',
  'Afghanistan',
  652230,
  652230,
  0,
  32564342,
  2.32,
  38.57,
  13.89,
  1.51,
  '2015-11-01 13:19:49.461734',
  '2015-11-01 13:19:49.461734'),
 (2,
  'al',
  'Albania',
  28748,
  27398,
  1350,
  3029278,
  0.3,
  12.92,
  6.58,
  3.3,
  '2015-11-01 13:19:54.431082',
  '2015-11-01 13:19:54.431082')]

# COUNT 
To count the number of records in a table using python, we would connect to the database, fetch all the records on the table we are intereted in as a list and the use len() function to count the number of records in that table as shown below.

facts = conn.execute("SELECT * FROM facts").fetchall()

facts_count = len(facts)

This approach is uses more code and is fairly as fetches all the records in the table before using the len(). SQL COUNT aggregation function can be used to this in a more efficient way as shown below.

SELECT COUNT(*) FROM facts;

In [35]:
conn.execute('SELECT COUNT(birth_rate) FROM facts').fetchone()[0]

228

In [40]:
# python version 
birth_rate = conn.execute("SELECT birth_rate FROM facts WHERE birth_rate IS NOT NULL").fetchall() 
len(birth_rate)

228

Note that to get the same result in python version a WHERE clause to filter NULL values is used, which is not require in COUNT aggregate function. COUNT is smart enough to exclude missing values.

In [43]:
# whithout WHERE clause
len(conn.execute("SELECT birth_rate FROM facts").fetchall())

261

# MAX, MIN, AVG, SUM
SQL has other aggregation functions, in addition to COUNT. MIN and MAX, for example, find the minimum and maximum values in a column. While SUM and AVG can be used to get the sum and average values of data in a column.

In [44]:
# MIN
conn.execute('SELECT MIN(population_growth) FROM facts WHERE population_growth !=""').fetchone()[0]

0.0

In [45]:
# MAX
conn.execute('SELECT MAX(death_rate) FROM facts WHERE death_rate !=""').fetchone()[0]

14.89

In [46]:
# SUM 
conn.execute('SELECT SUM(area_land) FROM facts WHERE area_land != ""').fetchone()[0]

128584834

In [47]:
# AVG
conn.execute('SELECT AVG(area_water) FROM facts WHERE area_water != ""').fetchone()[0]

19067.59259259259

Write a single query that calculates the following statistics about the facts table:
 - The mean of the population column, assign to mean_pop.
 - The sum of the population column, assign to sum_pop.
 - The maximum value in the birth_rate column, max_birth_rate.

In [48]:
result = conn.execute("SELECT AVG(population), SUM(population), MAX(birth_rate) FROM facts").fetchall()
mean_pop = result[0][0]
sum_pop = result[0][1]
max_birth_rate = result[0][2]

In [50]:
mean_pop

62094928.32231405

# DISTINCT
The SQL DISTINCT clause is used to remove duplicates from the result set of a SELECT statement

SELECT DISTINCT expressions
FROM tables
[WHERE conditions];

In [53]:
conn.execute("SELECT DISTINCT birth_rate FROM facts LIMIT 20").fetchall()

[(38.57,),
 (12.92,),
 (23.67,),
 (8.13,),
 (38.78,),
 (15.85,),
 (16.64,),
 (13.61,),
 (12.15,),
 (9.41,),
 (15.5,),
 (13.66,),
 (21.14,),
 (11.87,),
 (10.7,),
 (11.41,),
 (24.68,),
 (36.02,),
 (17.78,),
 (22.76,)]

In [54]:
# Using DISTICNT and Aggregate function
conn.execute("SELECT AVG(DISTINCT birth_rate) FROM facts WHERE population > 20000000").fetchone()[0]

20.43473684210527

# USing SQL to perform calcualtion
To calculate the ratio between births and deaths for each country, for example, we could divide the birth_rate column by the death_rate column. Here's how we would accomplish this:

SELECT birth_rate / death_rate 
FROM facts;

We can also perform more complex queries, such as finding the ratio of birth_rate plus migration_rate to death_rate. The results will help us discover whether the population is increasing or decreasing:

SELECT (birth_rate + migration_rate) / death_rate 
FROM facts;



In [55]:
next_year_population = conn.execute("SELECT (1 + population_growth/100)*population FROM facts").fetchall()

In [57]:
next_year_population[0:10]

[(33319834.734400004,),
 (3038365.834,),
 (40269741.8544,),
 (85682.69600000001,),
 (20170937.8134,),
 (93582.2064,),
 (43835802.5398,),
 (3060966.5730000003,),
 (22994449.849799998,),
 (8713210.525,)]

# Group Summary Statistics
Previously we computed summary statistics across columns with SQL. In many cases, though, we want to drill down even more and compute summary statistics per group. The SQL GROUP BY clause can be used in a SELECT statement to collect data across multiple records and group the results by one or more columns.

The syntax for the GROUP BY clause in SQL is:

SELECT expression1, expression2, ... expression_n, 
       aggregate_function (aggregate_expression)
FROM tables
[WHERE conditions]
GROUP BY expression1, expression2, ... expression_n
[ORDER BY expression [ ASC | DESC ]];

For example to fetch average ShareWomen for each Major_category the code below can be used

In [60]:
conn = sqlite3.connect("jobs.db")
conn.execute('''
    SELECT Major_category, AVG(ShareWomen) 
    FROM recent_grads
    GROUP BY Major_category  
''').fetchall()

[('Agriculture & Natural Resources', 0.6179384232),
 ('Arts', 0.56185119575),
 ('Biology & Life Science', 0.584518475857143),
 ('Business', 0.4050631853076923),
 ('Communications & Journalism', 0.64383484025),
 ('Computers & Mathematics', 0.5127519954545455),
 ('Education', 0.6749855163125),
 ('Engineering', 0.2571578951034483),
 ('Health', 0.6168565694166667),
 ('Humanities & Liberal Arts', 0.6761934042),
 ('Industrial Arts & Consumer Services', 0.4493512688571429),
 ('Interdisciplinary', 0.495397153),
 ('Law & Public Policy', 0.3359896912),
 ('Physical Sciences', 0.5087494197),
 ('Psychology & Social Work', 0.7777631628888888),
 ('Social Science', 0.5390672957777778)]

Write a query that selects the following items, in order, and renames them with AS:
 - SUM(Men) as total_men.
 - SUM(Women) as total_women
 
SELECT SUM(men) AS total_men, SUM(Women) AS total_women 
FROM recent_grads;

In [66]:
result = conn.execute('''
    SELECT SUM(men), SUM(Women)
    FROM recent_grads 
''').fetchone()
total_men,total_women = result

In [67]:
total_men

2878263

In [68]:
total_women

3897752

# HAVING clause
The SQL HAVING clause is used in combination with the GROUP BY clause to restrict the groups of returned rows to only those whose the condition is TRUE.

The syntax for the HAVING clause in SQL is:

 SELECT expression1, expression2, ... expression_n, 
       aggregate_function (aggregate_expression)
 FROM tables
 [WHERE conditions]
 GROUP BY expression1, expression2, ... expression_n
 HAVING condition;
 
Example HAVING clause can be used to Find all of the major categories where the share of graduates with low-wage jobs is greater than 0.1 as shown below.

In [69]:
conn.execute('''
    SELECT Major_category, AVG(Low_wage_jobs) / AVG(Total) as share_low_wage
    FROM recent_grads
    GROUP BY Major_category
    HAVING share_low_wage > 0.1
''').fetchall()

[('Arts', 0.16833085991095678),
 ('Communications & Journalism', 0.1263241815481876),
 ('Humanities & Liberal Arts', 0.13208721344194835),
 ('Industrial Arts & Consumer Services', 0.11571334076033978),
 ('Law & Public Policy', 0.11568503743572278),
 ('Psychology & Social Work', 0.11693384919554187),
 ('Social Science', 0.10223297343603174)]

# ROUND
SQL ROUND function can be used to round result.

In [70]:
# ShareWomen rounded to 4 decimal places
conn.execute('''
    SELECT ROUND(ShareWomen, 4),Major_category 
    FROM recent_grads 
    LIMIT 10;
''').fetchall()

[(0.1206, 'Engineering'),
 (0.1019, 'Engineering'),
 (0.153, 'Engineering'),
 (0.1073, 'Engineering'),
 (0.3416, 'Engineering'),
 (0.145, 'Engineering'),
 (0.5357, 'Business'),
 (0.4414, 'Physical Sciences'),
 (0.1398, 'Engineering'),
 (0.4378, 'Engineering')]

In [71]:
# more complex query with ROUND function
conn.execute('''
    SELECT Major_category, ROUND(AVG(College_jobs) / AVG(Total), 3) AS share_degree_jobs
    FROM recent_grads
    GROUP BY Major_category
    HAVING share_degree_jobs < 0.3
''').fetchall()

[('Agriculture & Natural Resources', 0.248),
 ('Arts', 0.265),
 ('Business', 0.114),
 ('Communications & Journalism', 0.22),
 ('Humanities & Liberal Arts', 0.27),
 ('Industrial Arts & Consumer Services', 0.249),
 ('Law & Public Policy', 0.163),
 ('Social Science', 0.215)]

# Challenge: Data Exploration
Write a single query that returns the following minimum and maximum values for countries where population is less than 2 billion and population is greater than 0:
 - Assign the minimum of the population column to pop_min.
 - Assign the maximum of the population column to pop_max.
 - Assign the minimum of the population_growth column to pop_growth_min.
 - Assign the maximum of the population_growth column to pop_growth_max.
 - Assign the minimum of the birth_rate column to birth_rate_min.
 - Assign the maximum of the birth_rate column to birth_rate_max.
 - Assign the minimum of the death_rate column to death_rate_min.
 - Assign the maximum of the death_rate column to death_rate_max.

In [72]:
# Solution
conn = sqlite3.connect("factbook.db")
query = "select min(population), max(population),min(population_growth), max(population_growth),min(birth_rate), max(birth_rate),min(death_rate), max(death_rate),min(migration_rate),max(migration_rate) from facts where population < 2000000000 and population > 0;"
results = conn.execute(query).fetchone()
pop_min = results[0]
pop_max = results[1]
pop_growth_min = results[2]
pop_growth_max = results[3]
birth_rate_min = results[4]
birth_rate_max = results[5]
dhttp://localhost:8888/notebooks/SQL_python.ipynb#Challenge-2eath_rate_min = results[6]
death_rate_max = results[7]

In [73]:
pop_max

1367485388

# Challenge 2
projected_population = population + (population * (population_growth/100))
Use SQL arithmetic to return the projected population values using the above formula and the following parameters:
 - Round the values to the nearest whole number (population can't contain a fractional value).
 - Filter out any rows with NULL as the value for either population or population_growth.
 - Restrict the query to countries with a population that's less than 7 billion and greater than 0.
 - Assign the resulting projections to projected_population.

In [79]:
query = '''
SELECT ROUND(population + population * (population_growth/100), 0) FROM facts
WHERE population > 0 AND population < 7000000000 
AND population IS NOT NULL AND population_growth IS NOT NULL;
'''
projected_population = conn.execute(query).fetchall()

In [78]:
projected_population[0:20]

[(33319835.0,),
 (3038366.0,),
 (40269742.0,),
 (85683.0,),
 (20170938.0,),
 (93582.0,),
 (43835803.0,),
 (3060967.0,),
 (22994450.0,),
 (8713211.0,),
 (9874675.0,),
 (327356.0,),
 (1379066.0,),
 (171661069.0,),
 (291505.0,),
 (9608868.0,),
 (11410035.0,),
 (353865.0,),
 (10739119.0,),
 (750154.0,)]

 # Challenge 3
Write a single query that returns:
 - the minimum of the projected population values, and assigns it to pop_proj_min.
 - the maximum of the projected population values, and assigns it to pop_proj_max.
 - the average of the projected population values, and assigns it to pop_proj_avg.

Be sure to:
 - Round all fractional values to the nearest whole number.
 - Filter out any rows with NULL as the value for either population or population_growth.
 - Restrict the query to countries with a population of less than 7 billion and greater than 0.
 - Use print statements or the variables display below the output box to observe these values.   

In [80]:
query = '''
SELECT MIN(ROUND(population + population * (population_growth/100), 0)), 
MAX(ROUND(population + population * (population_growth/100), 0)),
AVG(round(population + population * (population_growth/100), 0)) from facts
WHERE population > 0 AND population < 7000000000 
AND population IS NOT NULL AND population_growth IS NOT NULL;
'''
projected_population = conn.execute(query).fetchone()
pop_proj_min = projected_population[0]
pop_proj_max = projected_population[1]
pop_proj_avg = projected_population[2]

In [81]:
pop_proj_min

48.0